# Load dataset

In [2]:
from datasets import list_datasets, load_dataset
dataset = load_dataset("pubmed_qa", "pqa_labeled",split="train")
print(dataset)

/Users/giangnguyen/RMIT/capstone/Data-Clean/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset pubmed_qa (/Users/giangnguyen/.cache/huggingface/datasets/pubmed_qa/pqa_labeled/1.0.0/dd4c39f031a958c7e782595fa4dd1b1330484e8bbadd4d9212e5046f27e68924)


Dataset({
    features: ['pubid', 'question', 'context', 'long_answer', 'final_decision'],
    num_rows: 1000
})


In [3]:
dataset[0]

{'pubid': 21645374,
 'question': 'Do mitochondria play a role in remodelling lace plant leaves during programmed cell death?',
 'context': {'contexts': ['Programmed cell death (PCD) is the regulated death of cells within an organism. The lace plant (Aponogeton madagascariensis) produces perforations in its leaves through PCD. The leaves of the plant consist of a latticework of longitudinal and transverse veins enclosing areoles. PCD occurs in the cells at the center of these areoles and progresses outwards, stopping approximately five cells from the vasculature. The role of mitochondria during PCD has been recognized in animals; however, it has been less studied during PCD in plants.',
   'The following paper elucidates the role of mitochondrial dynamics during developmentally regulated PCD in vivo in A. madagascariensis. A single areole within a window stage leaf (PCD is occurring) was divided into three areas based on the progression of PCD; cells that will not undergo PCD (NPCD), ce

## Use third-party libs for translation & openai setup

In [4]:
import os
from dotenv import load_dotenv
import translators as ts
import openai
load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")

Using region Ho Chi Minh server backend.



In [5]:
def build_prompt(text, context):
    translated_text= ts.translate_text(text, from_language='en', to_language='vi')
    print(translated_text)
    translated_context =ts.translate_text(text, from_language='en', to_language='vi')
    return [
        {"role": "system", "content": """
         User will provide you a context, an a text that you will translate and a Literary translation with no context.
         You will do the following step:
         Step 1: Read the context
         Step 2: Use the sample Literary translation and the context for reference.
         Step 3: Translate the input text using the context and sample translation with no context
         Step 4: Return out after being translated only.
         """},
        {"role": "user", "content": f"""
Context: {context}
Literary translation: {translated_text}
Input: {text}
Output: """}]


def translate(messages):
    try:
        chat_completion = openai.ChatCompletion.create(model="gpt-3.5-turbo",
                                                   messages=messages)
        print(f'{chat_completion["usage"]["prompt_tokens"]} prompt tokens used.')
        return chat_completion.choices[0]["message"].content
    except Exception as error:
        print("Error: "+str(error))
    

## Sample translation

In [6]:
# messages = build_prompt(dataset[0]["long_answer"], dataset[0]["context"]["contexts"][0])
# output = translate(messages)
# output

## Multi-thread translation

Note: 1 single request seems to work but spamming the server (this libs uses Bing translate) return `too many requests error`

In [14]:
from concurrent.futures import ThreadPoolExecutor
import pandas as pd
from datetime import datetime
translated_questions=[]
translated_answers=[]

for i in range(min(len(dataset), 100)):
    try:
        question = dataset[i]["question"]
        answer = dataset[i]["long_answer"]
        translated_question = ts.translate_text(question, from_language='en', to_language='vi')
        translated_answer = ts.translate_text(answer, from_language='en', to_language='vi')
        print('successsss 🎊')
        translated_questions.append(translated_question)
        translated_answers.append(translated_answer)
    except:
        translated_questions.append("Error")
        translated_answers.append("Error")
dataset_1=dataset.select(range(min(len(dataset), 100)))

dataset_1 = dataset_1.add_column("translated_question", translated_questions)
dataset_1 = dataset_1.add_column("translated_answers", translated_answers)

df = pd.DataFrame(dataset_1)
df.to_csv("translated_pubmedqa_1.csv", index=False)


successsss 🎊
successsss 🎊
successsss 🎊
successsss 🎊
successsss 🎊
successsss 🎊
successsss 🎊
successsss 🎊
successsss 🎊
successsss 🎊
successsss 🎊
successsss 🎊
successsss 🎊
successsss 🎊
successsss 🎊
successsss 🎊
successsss 🎊
successsss 🎊
successsss 🎊
successsss 🎊
successsss 🎊
successsss 🎊
successsss 🎊
successsss 🎊
